In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install torch

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import random

In [ ]:
import json
with open("/content/drive/MyDrive/Disertatie/Lyrics_ExtractTopics_12.06.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [ ]:
texts = []
themes = []
for song in data:
    theme = song.get("theme", "unknown")
    lyrics = song["lyrics"].replace("\n", " ").strip()
    if lyrics:
        texts.append(f"{theme.lower()} {lyrics.lower()}")
        themes.append(theme.lower())

In [ ]:
vocab_size = 10000
max_len = 30
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)

input_sequences = []
for line in texts:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [ ]:
max_seq_len = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_seq_len, padding='pre')
x, y = input_sequences[:,:-1], input_sequences[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=vocab_size)

In [ ]:
print(max_seq_len)

2204


In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=max_seq_len - 1))
model.add(LSTM(128, implementation=2))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tqdm.auto import tqdm
from tensorflow.keras.callbacks import Callback
import time

class CustomProgressBar(Callback):
    def on_train_begin(self, logs=None):
        self.epochs = self.params.get('epochs', 0)
        self.steps = self.params.get('steps', 0)
        self.epoch_bar = tqdm(total=self.epochs, desc="Training", position=0)

    def on_epoch_begin(self, epoch, logs=None):
        self.step_bar = tqdm(total=self.steps, desc=f"Epoch {epoch+1}", position=1, leave=False)
        self.epoch_start = time.time()

    def on_train_batch_end(self, batch, logs=None):
        self.step_bar.update(1)

    def on_epoch_end(self, epoch, logs=None):
        self.step_bar.close()
        duration = time.time() - self.epoch_start
        self.epoch_bar.set_postfix_str(f"Epoch time: {duration:.2f}s")
        self.epoch_bar.update(1)

    def on_train_end(self, logs=None):
        self.epoch_bar.close()

In [ ]:
print(x.shape)
print(x.dtype)

(256904, 2203)
int32


In [ ]:
x = x.astype('float32')

In [ ]:
model.fit(x, y, epochs=30, batch_size=64, callbacks=[CustomProgressBar()])

Training:   0%|          | 0/30 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/4015 [00:00<?, ?it/s]

Epoch 1/30
4015/4015 ━━━━━━━━━━━━━━━━━━━━ 382s 94ms/step - accuracy: 0.0521 - loss: 6.6392


Epoch 2:   0%|          | 0/4015 [00:00<?, ?it/s]

Epoch 2/30
4015/4015 ━━━━━━━━━━━━━━━━━━━━ 388s 97ms/step - accuracy: 0.1125 - loss: 5.7004


...

Epoch 28/30
4015/4015 ━━━━━━━━━━━━━━━━━━━━ 388s 97ms/step - accuracy: 0.6420 - loss: 1.7038


Epoch 29:   0%|          | 0/4015 [00:00<?, ?it/s]

Epoch 29/30
4015/4015 ━━━━━━━━━━━━━━━━━━━━ 388s 97ms/step - accuracy: 0.6476 - loss: 1.6674


Epoch 30:   0%|          | 0/4015 [00:00<?, ?it/s]

Epoch 30/30
4015/4015 ━━━━━━━━━━━━━━━━━━━━ 388s 97ms/step - accuracy: 0.6541 - loss: 1.6354


In [ ]:
model.save("/content/drive/MyDrive/lstm_kendrick_model.h5")
with open("/content/drive/MyDrive/tokenizer_kendrick.pkl", "wb") as f:
    pickle.dump(tokenizer, f)